In [150]:
from keras.layers import Input, Dense, Conv2D, Add, Dot, Lambda, Conv2DTranspose, Dot, Activation, Reshape, BatchNormalization, UpSampling2D, AveragePooling2D, GlobalAveragePooling2D, Multiply, LeakyReLU, Flatten, MaxPool2D 
from keras.models import Model
import keras.backend as K

In [14]:
def ResBlockDown(input_shape, channel_size, channel_multiplier=1, name=None):
    # Resblock architecture
    # 1 BatchNorm 
    # 2 ReLU activation
    # 3 Conv layer
    # 4 BatchNorm
    # 5 ReLU activation
    # 6 Conv layer
    # 7 Sum with input 
    
    #FIRST BLOCK
    #input layer
    input_layer = Input(shape=input_shape)
    
    # BatchNorm - needs to be conditional
    resblock = BatchNormalization()(input_layer)
    
    # Relu
    resblock = Activation('relu')(resblock)
    
    # Convolution size 3 filter as per paper
    # Need to spectrally normalize here somehow
    resblock = Conv2D(channel_size * channel_multiplier, 3, padding='same')(resblock)
    
    #SECOND BLOCK
    
    # BatchNorm - needs to be conditional
    resblock = BatchNormalization()(input_layer)
    
    # Relu
    resblock = Activation('relu')(resblock)
    
    # Convolution size 3 filter as per paper
    # Need to spectrally normalize here somehow
    resblock = Conv2D(channel_size * channel_multiplier, 3, padding='same')(resblock)
    
    # Downsample
    resblock = AveragePooling2D()(resblock)
    
    # Time for the shortcut connection!
    
    shortcut_identity = Conv2D(channel_size * channel_multiplier, 1, padding='same')(input_layer)
    shortcut_identity = AveragePooling2D()(shortcut_identity)
    
    output_layer = Add()([shortcut_identity, resblock])
    
    return Model(input_layer, output_layer, name=name)
    
    
    
    
    

In [19]:
def ResBlock(input_shape, channel_size, channel_multiplier=1, name=None):
     # Resblock architecture
    # 1 BatchNorm 
    # 2 ReLU activation
    # 3 Conv layer
    # 4 BatchNorm
    # 5 ReLU activation
    # 6 Conv layer
    # 7 Sum with input 
    
    #FIRST BLOCK
    #input layer
    input_layer = Input(shape=input_shape)
    
    # BatchNorm - needs to be conditional
    resblock = BatchNormalization()(input_layer)
    
    # Relu
    resblock = Activation('relu')(resblock)
    
    # Convolution size 3 filter as per paper
    # Need to spectrally normalize here somehow
    resblock = Conv2D(channel_size * channel_multiplier, 3, padding='same')(resblock)
    
    #SECOND BLOCK
    
    # BatchNorm - needs to be conditional
    resblock = BatchNormalization()(input_layer)
    
    # Relu
    resblock = Activation('relu')(resblock)
    
    # Convolution size 3 filter as per paper
    # Need to spectrally normalize here somehow
    resblock = Conv2D(channel_size * channel_multiplier, 3, padding='same')(resblock)
    
    
    # Time for the shortcut connection!
    
    shortcut_identity = Conv2D(channel_size * channel_multiplier, 1, padding='same')(input_layer)
    
    output_layer = Add()([shortcut_identity, resblock])
    
    return Model(input_layer, output_layer, name=name)
    
    
    
    
    
    

In [168]:


def SelfAttentionBlock(input_shape, name=None):
    # f = conv
    channels = input_shape[-1]
    input_layer = Input(shape=input_shape)
    f = Conv2D(channels // 8, 1, padding='same')(input_layer)
    # f = maxpooling
    f = MaxPool2D(pool_size=2, strides=2, padding='same')(f)
    
    g = Conv2D(channels // 8, 1, padding='same')(input_layer)
    
    h = Conv2D(channels // 2, 1, padding='same')(input_layer)
    h = MaxPool2D(pool_size=2, strides=2, padding='same')(h)
    
    
    g = Lambda(lambda input1: K.reshape(input1, shape=[-1,1]))(g)
    f = Lambda(lambda input1: K.reshape(input1, shape=[-1,1]))(f)
    s = Dot(-1)([g, f])
    beta = Activation('softmax')(s)

    h = Lambda(lambda input1: K.reshape(input1, shape=[-1,1]))(h)
    o = Dot(-1)([beta, h])
    
    gamma = Conv2D(channels, (1, 1), padding='same', use_bias=False, kernel_initializer='he_normal')(input_layer)
    #gamma = Reshape((-1, channels // 2))(g)
    #print(gamma.shape)
    a, x, y ,z = input_layer.shape
    #o = K.reshape(o, shape=[x,y,z, channels // 2])
    o = Lambda(lambda input1: K.reshape(input1, shape=[x,y,z, channels // 2]))(o)
    o = Conv2D(channels, kernel_size=1, strides=1)(o)
  
    Wz_yi = Multiply()([gamma, o])
    output_layer = Add()([Wz_yi, input_layer])
    
    return Model(input_layer, output_layer, name=name)
    
    # g = conv
    # h = conv
    # h = maxpooling

In [169]:
class GlobalSumPooling2D(_GlobalPooling2D):
    """Global sum pooling operation for spatial data.
    # Arguments
        data_format: A string,
            one of `channels_last` (default) or `channels_first`.
            The ordering of the dimensions in the inputs.
            `channels_last` corresponds to inputs with shape
            `(batch, height, width, channels)` while `channels_first`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be "channels_last".
    # Input shape
        - If `data_format='channels_last'`:
            4D tensor with shape:
            `(batch_size, rows, cols, channels)`
        - If `data_format='channels_first'`:
            4D tensor with shape:
            `(batch_size, channels, rows, cols)`
    # Output shape
        2D tensor with shape:
        `(batch_size, channels)`
    """
    def call(self, inputs):
        if self.data_format == 'channels_last':
            return K.sum(inputs, axis=[1, 2])
        else:
            return K.sum(inputs, axis=[2, 3])

# Discriminator test
model_input = Input(shape=(128,128,3))
resblockdown1 = ResBlockDown(input_shape=(128,128,3),channel_size=1, channel_multiplier=64, name='Discriminator_resblock_down_1')
h = resblockdown1(model_input)
print(h)
selfattentionblock = SelfAttentionBlock(input_shape=(64,64,64), name='Discriminator_selfattentionblock')
h = selfattentionblock()(h)
print(h)
# Non local block should be here
resblockdown2 = ResBlockDown(input_shape=(64,64,64),channel_size=2, channel_multiplier=64, name='Discriminator_resblock_down_2')
h = resblockdown2(h)
print(h)
resblockdown4 = ResBlockDown(input_shape=(32,32,128),channel_size=4, channel_multiplier=64, name='Discriminator_resblock_down_4')
h = resblockdown4(h)
print(h)
resblockdown8 = ResBlockDown(input_shape=(16,16,256),channel_size=8, channel_multiplier=64, name='Discriminator_resblock_down_8')
h = resblockdown8(h)
print(h)
resblockdown16 = ResBlockDown(input_shape=(8,8,512),channel_size=16, channel_multiplier=64, name='Discriminator_resblock_down_16')
h = resblockdown16(h)
print(h)
resblock16 = ResBlock(input_shape=(4,4,1024),channel_size=16, channel_multiplier=64, name='Discriminator_resblock_16')
h = resblock16(h)
print(h)
h = Activation('relu')(h)
print(h)
h = GlobalSumPooling2D()(h)
model_output = Dense(1)(h)
print(h)
# we need an embedding layer instead of dense
model = Model(model_input, model_output, name="Sonofabish")

Tensor("Discriminator_resblock_down_1_155/add_243/add:0", shape=(?, 64, 64, 64), dtype=float32)


TypeError: __call__() missing 1 required positional argument: 'inputs'